### DSE 220 Final: LSTM Neural Network Model for Amazon Review Helpfulness

In [3]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from six.moves import cPickle as pickle
import pandas as pd
import gzip
import seaborn as sns
import string
from time import time
import nltk
from nltk.corpus import stopwords 
#stops = set(stopwords.words("english"))

In [4]:
# Import the classifier
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap         # Visualization
import numpy as np                                   # Numerical operations
import matplotlib.pyplot as plt                      # Plotting
from sklearn.model_selection import train_test_split # Data splitting
import pandas as pd
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import datasets, decomposition, feature_extraction
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2
from nltk.corpus import brown
import collections
import operator
import math
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from nltk.cluster import KMeansClusterer, euclidean_distance, cosine_distance
from sklearn.preprocessing import StandardScaler

In [5]:
import gzip
from collections import defaultdict

## Helpfulness baseline 
### Compute the global average helpfulness rate, and the average helpfulness rate for each user

In [6]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

allHelpful = []
userHelpful = defaultdict(list)

for l in readGz("train.json.gz"):
  user,item = l['reviewerID'],l['itemID']
  allHelpful.append(l['helpful'])
  userHelpful[user].append(l['helpful'])

averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}
for u in userHelpful:
  totalU = sum([x['outOf'] for x in userHelpful[u]])
  if totalU > 0:
    userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / totalU
  else:
    userRate[u] = averageRate

predictions = open("predictions_Helpful.txt", 'w')
for l in open("pairs_Helpful.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i,outOf = l.strip().split('-')
  outOf = int(outOf)
  if u in userRate:
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*userRate[u]) + '\n')
  else:
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')

predictions.close()

In [7]:
averageRate

0.8519720886532813

In [8]:
userHelpful

defaultdict(list,
            {'U745881038': [{'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 2, 'nHelpful': 2}],
             'U023577405': [{'outOf': 0, 'nHelpful': 0},
              {'outOf': 6, 'nHelpful': 6},
              {'outOf': 1, 'nHelpful': 1},
              {'outOf': 1, 'nHelpful': 1},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0}],
             'U441384838': [{'outOf': 2, 'nHelpful': 2},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 5, 'nHelpful': 3},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 5, 'nHelpful': 5}],
             'U654041297': [{'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0},
              {'outOf': 0, 'nHelpful': 0}],
             'U096604734': [{'outOf': 1, 'nHelpful': 1},
     

## For reading the files in a pandas Dataframe.

In [9]:
import pandas as pd
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('train.json.gz')
test_df = getDF('test_Helpful.json.gz')

In [10]:
df.head(5)

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,helpful,price
0,0,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",I655355328,U745881038,3.0,"These are cute, but they are a little small. ...",R115160670,"05 20, 2014",Cute,1400544000,"{'outOf': 0, 'nHelpful': 0}",NaN
1,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I241092314,U023577405,4.0,"I love the look of this bra, it is what I want...",R800651687,"02 7, 2013",Beautiful but size runs small,1360195200,"{'outOf': 0, 'nHelpful': 0}",NaN
2,0,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",I408260822,U441384838,3.0,it's better on a man's hand.I didn't find it v...,R345042616,"05 13, 2014",Good price but...,1399939200,"{'outOf': 2, 'nHelpful': 2}",19.99
3,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I770448753,U654041297,4.0,Comfortable and easy to wear for a day of shop...,R875466866,"05 25, 2014","Easy, breezy",1400976000,"{'outOf': 0, 'nHelpful': 0}",14.95
4,0,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",I919238161,U096604734,5.0,I'm quite small and the XS fits me like a regu...,R317526520,"07 30, 2013",Great shirt,1375142400,"{'outOf': 1, 'nHelpful': 1}",NaN


In [11]:
df.shape

(200000, 12)

In [12]:
test_df.head()

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,helpful,price
0,0,"[[Sports & Outdoors, Other Sports, Dance, Clot...",I520932398,U816789534,3.0,I ordered according to the size chart but it's...,R157684793,"07 15, 2011",Too small,1310688000,{'outOf': 2},NaN
1,0,"[[Sports & Outdoors, Clothing, Women, Hoodies]...",I969532331,U987148846,4.0,Super thin but really cute and not cheap-looki...,R732719858,"07 17, 2013",Fun hoodie,1374019200,{'outOf': 0},NaN
2,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I149943341,U628436634,5.0,"It was a present for my sis, and she loves Fle...",R352659313,"12 8, 2013",A Perfect hook up,1386460800,{'outOf': 1},NaN
3,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I909025835,U924107228,5.0,I love this thing! I guess they don't make th...,R277416618,"11 22, 2012",I love this thing...,1353542400,{'outOf': 1},7.51
4,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I228439768,U060135484,4.0,I liked it and I wear it...it's a little bit s...,R645892076,"04 1, 2014",I liked it...,1396310400,{'outOf': 0},NaN


In [13]:
test_df.shape

(14000, 12)

In [14]:
#1. Define function to get the first review time for each item
def first_review_time(data_df):
    time_dict = {}
    for i in range(len(data_df)):
        pid = data_df['itemID'][i]
        #print (pid)
        time_i = data_df['reviewTime'][i]
        if pid in time_dict:
            if time_i < time_dict[pid]:
                time_dict[pid] = time_i
        else:
            time_dict[pid] = time_i
    data_df['firstReviewTime'] = data_df['itemID'].map(time_dict).values
    return data_df
# convert reviewTime to datatime data type
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df = first_review_time(df)
df['review_first_dif'] = (df['reviewTime'] - df['firstReviewTime']).astype('timedelta64[D]')
test_df['reviewTime'] = pd.to_datetime(test_df['reviewTime'])
test_df = first_review_time(test_df)
test_df['review_first_dif'] = (test_df['reviewTime'] - test_df['firstReviewTime']).astype('timedelta64[D]')

In [15]:
#2. Rating score deviation from mean
def deviation_mean(data_df):
    rating_mean_dict = data_df['rating'].groupby(data_df['itemID']).mean().to_dict()
    data_df['rating_mean'] = data_df['itemID'].map(rating_mean_dict).values
    data_df['rating_mean_dev'] = data_df['rating'] - data_df['rating_mean']
    return data_df['rating_mean_dev']
df['rating_mean_dev'] = deviation_mean(df)
test_df['rating_mean_dev'] = deviation_mean(test_df)

In [16]:
#3. Number of words of each review text
df['reviewWords'] = df['reviewText'].apply(lambda x: len(x.split()))
test_df['reviewWords'] = test_df['reviewText'].apply(lambda x: len(x.split()))

In [17]:
#4. Number of words of each review summary
df['summaryWords'] = df['summary'].apply(lambda x: len(x.split()))
test_df['summaryWords'] = test_df['summary'].apply(lambda x: len(x.split()))
#df.head(5)

In [18]:
#5. Ratio of summary words to review text words
df['ratiosuWord'] = df['summaryWords'] / df['reviewWords']
test_df['ratiosuWord'] = test_df['summaryWords'] / test_df['reviewWords']

In [19]:
#6. Number of sentences of each review text
def count_sentence(data_df, text):
    pun_sen = ['.', '!', '?']
    text_col = data_df[text]
    sentence_counts = []
    for i in text_col:
        sentence_count = []
        for j in pun_sen:
            count_a = i.count(j)
            sentence_count.append(count_a)
        sentence_counts.append(sum(sentence_count))
    data_df['reviewSentences'] = sentence_counts
    return data_df['reviewSentences']
df['reviewSentences'] = count_sentence(df, 'reviewText')
test_df['reviewSentences'] = count_sentence(test_df, 'reviewText')

In [20]:
#extract number of sentences of each review text
df['summarySentences'] = count_sentence(df, 'summary')
test_df['summarySentences'] = count_sentence(test_df, 'summary')
#summarySentences = []
#for i in df['summary']:
#    j = i.count('.')
#    summarySentences.append(j)
#df['summarysentences'] = summarySentences
df.head()

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,...,price,firstReviewTime,review_first_dif,rating_mean,rating_mean_dev,reviewWords,summaryWords,ratiosuWord,reviewSentences,summarySentences
0,0,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",I655355328,U745881038,3.0,"These are cute, but they are a little small. ...",R115160670,2014-05-20,Cute,1400544000,...,NaN,2013-10-30,202.0,4.055556,-1.055556,24,1,0.041667,0,0
1,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I241092314,U023577405,4.0,"I love the look of this bra, it is what I want...",R800651687,2013-02-07,Beautiful but size runs small,1360195200,...,NaN,2012-06-06,246.0,3.555556,0.444444,57,5,0.087719,0,0
2,0,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",I408260822,U441384838,3.0,it's better on a man's hand.I didn't find it v...,R345042616,2014-05-13,Good price but...,1399939200,...,19.99,2013-01-13,485.0,4.083333,-1.083333,28,3,0.107143,3,3
3,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I770448753,U654041297,4.0,Comfortable and easy to wear for a day of shop...,R875466866,2014-05-25,"Easy, breezy",1400976000,...,14.95,2013-12-08,168.0,4.142857,-0.142857,31,2,0.064516,0,0
4,0,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",I919238161,U096604734,5.0,I'm quite small and the XS fits me like a regu...,R317526520,2013-07-30,Great shirt,1375142400,...,NaN,2012-12-24,218.0,4.500000,0.500000,77,2,0.025974,0,0


In [21]:
#7. Number of characters of each review text
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
def count_characters(data_df):
    reviewcharacters = []
    text_col = data_df['reviewText']
    for i in text_col:
        a = dict(collections.Counter(i))
        b = {k:v for k, v in a.items() if k not in punctuation}
        c = sum(list(b.values()))
        reviewcharacters.append(c)
    data_df['reviewChars'] = reviewcharacters
    return data_df['reviewChars']
df['reviewChars'] = count_characters(df)
test_df['reviewChars'] = count_characters(test_df)
#df.head(5)

In [22]:
#punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
#lowercase_words = [x.lower() for x in brown.words()]
#punctuation_stopwords = punctuation + stopwords.words('english')
#filtered_words = [x for x in lowercase_words if x not in punctuation_stopwords]
#final_filtered_words = list(filter(lambda x: x.isalpha() and len(x) > 1, filtered_words))

In [23]:
#8. Readability of each review (ARI as index to measure)
def readability(data_df):
    wordperSen = []
    charperWord = []
    reviewRead = []
    len_df = len(data_df)
    a = list(data_df['reviewWords'])
    b = list(data_df['reviewSentences'])
    c = list(data_df['reviewChars'])
    for i in range(len_df):
        if b[i] == 0:
            wordperSen.append(0)
        else:
            j = a[i] / b[i]
            wordperSen.append(j)
        if a[i] == 0:
            charperWord.append(0)
        else:
            l = c[i] / a[i]
            charperWord.append(l)
        ari = 4.71 * charperWord[i] + 0.5 * wordperSen[i] - 21.43
        reviewRead.append(ari)
    data_df['reviewRead'] = reviewRead
    return data_df['reviewRead']
df['reviewRead'] = readability(df)
test_df['reviewRead'] = readability(df)

In [24]:
#9. Number of punctuations of each review text
def numpunct(data_df):
    reviewPuncts = []
    for i in data_df['reviewText']:
        a = dict(collections.Counter(i))
        b = {k:v for k,v in a.items() if k in punctuation}
        c = sum(list(b.values()))
        reviewPuncts.append(c)
    data_df['reviewPuncts'] = reviewPuncts
    return data_df['reviewPuncts']
df['reviewPuncts'] = numpunct(df)
test_df['reviewPuncts'] = numpunct(test_df)

In [25]:
#10. Ratio of punctuations with characters
def ratio_puncts_chars(data_df):
    return data_df['reviewPuncts'] / data_df['reviewChars']
df['ratiopunChar'] = ratio_puncts_chars(df)
test_df['ratiopunChar'] = ratio_puncts_chars(test_df)

In [26]:
#11. Number of capital words of each review text
def numcapwords(data_df):
    reviewCwords = []
    for i in data_df['reviewText']:
        a = i.split()
        b = [word for word in a if word.isupper()]
        c = len(b)
        reviewCwords.append(c)
    data_df['reviewCwords'] = reviewCwords
    return data_df['reviewCwords']
df['reviewCwords'] = numcapwords(df)
test_df['reviewCwords'] = numcapwords(test_df)

In [27]:
#12. Number of capital words of each summary
def numcapwords(data_df):
    reviewCwords = []
    for i in data_df['summary']:
        a = i.split()
        b = [word for word in a if word.isupper()]
        c = len(b)
        reviewCwords.append(c)
    data_df['summaryCwords'] = reviewCwords
    return data_df['summaryCwords']
df['summaryCwords'] = numcapwords(df)
test_df['summaryCwords'] = numcapwords(test_df)

In [28]:
import re
#13. Number of exclimation and question marks of each review text
def numexclquest(data_df):
    suexcqueMarks = []
    for i in data_df['reviewText']:
        a = re.findall(r'[!?]', i)
        suexcqueMarks.append(len(a))
    data_df['numexclquest'] = suexcqueMarks
    return data_df['numexclquest']
df['numexclquest'] = numexclquest(df)
test_df['numexclquest'] = numexclquest(test_df)

In [29]:
#14. Number of exclimation and question marks of each summary text
def numexclquest(data_df):
    suexcqueMarks = []
    for i in data_df['summary']:
        a = re.findall(r'[!?]', i)
        suexcqueMarks.append(len(a))
    data_df['sunumexclquest'] = suexcqueMarks
    return data_df['sunumexclquest']
df['sunumexclquest'] = numexclquest(df)
test_df['sunumexclquest'] = numexclquest(test_df)

In [30]:
#15. Number of reviews of each product (measure the popularity of each product)
def numreviewPro(data_df):
    itemid_dict = data_df.groupby('itemID')['itemID'].count().to_dict()
    data_df['numreviewPro'] = data_df['itemID'].map(itemid_dict).values
    return data_df['numreviewPro']
df['numreviewPro'] = numreviewPro(df)
test_df['numreviewPro'] = numreviewPro(test_df)

In [31]:
#16. Number of reviews of each reviewers (measure reviewer’s experience)
def numreviewPro(data_df):
    itemid_dict = data_df.groupby('reviewerID')['reviewerID'].count().to_dict()
    data_df['numreviews'] = data_df['reviewerID'].map(itemid_dict).values
    return data_df['numreviews']
df['numreviews'] = numreviewPro(df)
test_df['numreviews'] = numreviewPro(test_df)

In [32]:
from textblob import TextBlob
import textstat

In [33]:
#16.1
df['summarypolarity'] = df['summary'].iloc[:].apply(lambda x: TextBlob(x).sentiment[0])
df['summarysubjectivity'] = df['summary'].iloc[:].apply(lambda x: TextBlob(x).sentiment[1])
df['reviewTextpolarity'] = df['reviewText'].iloc[:].apply(lambda x: TextBlob(x).sentiment[0])
df['reviewTextsubjectivity'] = df['reviewText'].iloc[:].apply(lambda x: TextBlob(x).sentiment[1])

test_df['summarypolarity'] = test_df['summary'].iloc[:].apply(lambda x: TextBlob(x).sentiment[0])
test_df['summarysubjectivity'] = test_df['summary'].iloc[:].apply(lambda x: TextBlob(x).sentiment[1])
test_df['reviewTextpolarity'] = test_df['reviewText'].iloc[:].apply(lambda x: TextBlob(x).sentiment[0])
test_df['reviewTextsubjectivity'] = test_df['reviewText'].iloc[:].apply(lambda x: TextBlob(x).sentiment[1])

In [35]:
#16.2
#df['reviewText_flesch_reading_ease'] = df['reviewText'].apply(lambda x: textstat.flesch_reading_ease(x))
#df['reviewText_smog_index'] = df['reviewText'].apply(lambda x: textstat.smog_index(x))
#df['reviewText_coleman_liau_index'] = df['reviewText'].apply(lambda x: textstat.coleman_liau_index(x))
#df['reviewText_automated_readability_index'] = df['reviewText'].apply(lambda x: textstat.automated_readability_index(x))
#df['reviewText_dale_chall_readability_score'] = df['reviewText'].apply(lambda x: textstat.dale_chall_readability_score(x))
#df['reviewText_difficult_word'] = df['reviewText'].apply(lambda x: textstat.difficult_words(x))
#df['reviewText_linsear_write_formula'] = df['reviewText'].apply(lambda x: textstat.linsear_write_formula(x))
#df['reviewText_gunning_fog'] = df['reviewText'].apply(lambda x: textstat.gunning_fog(x))
#df['reviewText_text_standard'] = df['reviewText'].apply(lambda x: textstat.text_standard(x))

#test_df['reviewText_flesch_reading_ease'] = test_df['reviewText'].apply(lambda x: textstat.flesch_reading_ease(x))
#test_df['reviewText_smog_index'] = test_df['reviewText'].apply(lambda x: textstat.smog_index(x))
#test_df['reviewText_coleman_liau_index'] = test_df['reviewText'].apply(lambda x: textstat.coleman_liau_index(x))
#test_df['reviewText_automated_readability_index'] = test_df['reviewText'].apply(lambda x: textstat.automated_readability_index(x))
#test_df['reviewText_dale_chall_readability_score'] = test_df['reviewText'].apply(lambda x: textstat.dale_chall_readability_score(x))
#test_df['reviewText_difficult_word'] = test_df['reviewText'].apply(lambda x: textstat.difficult_words(x))
#test_df['reviewText_linsear_write_formula'] = test_df['reviewText'].apply(lambda x: textstat.linsear_write_formula(x))
#test_df['reviewText_gunning_fog'] = test_df['reviewText'].apply(lambda x: textstat.gunning_fog(x))
#test_df['reviewText_text_standard'] = test_df['reviewText'].apply(lambda x: textstat.text_standard(x))

#df['summary_flesch_reading_ease'] = df['summary'].apply(lambda x: textstat.flesch_reading_ease(x))
#df['summary_smog_index'] = df['summary'].apply(lambda x: textstat.smog_index(x))
#df['summary_coleman_liau_index'] = df['summary'].apply(lambda x: textstat.coleman_liau_index(x))
#df['summary_automated_readability_index'] = df['summary'].apply(lambda x: textstat.automated_readability_index(x))
#df['summary_dale_chall_readability_score'] = df['summary'].apply(lambda x: textstat.dale_chall_readability_score(x))
#df['summary_difficult_word'] = df['summary'].apply(lambda x: textstat.difficult_words(x))
#df['summary_linsear_write_formula'] = df['summary'].apply(lambda x: textstat.linsear_write_formula(x))
#df['summary_gunning_fog'] = df['summary'].apply(lambda x: textstat.gunning_fog(x))
#df['summary_text_standard'] = df['summary'].apply(lambda x: textstat.text_standard(x))

#test_df['summary_flesch_reading_ease'] = test_df['summary'].apply(lambda x: textstat.flesch_reading_ease(x))
#test_df['summary_smog_index'] = test_df['summary'].apply(lambda x: textstat.smog_index(x))
#test_df['summary_coleman_liau_index'] = test_df['summary'].apply(lambda x: textstat.coleman_liau_index(x))
#test_df['summary_automated_readability_index'] = test_df['summary'].apply(lambda x: textstat.automated_readability_index(x))
#test_df['summary_dale_chall_readability_score'] = test_df['summary'].apply(lambda x: textstat.dale_chall_readability_score(x))
#test_df['summary_difficult_word'] = test_df['summary'].apply(lambda x: textstat.difficult_words(x))
#test_df['summary_linsear_write_formula'] = test_df['summary'].apply(lambda x: textstat.linsear_write_formula(x))
#test_df['summary_gunning_fog'] = test_df['summary'].apply(lambda x: textstat.gunning_fog(x))
#test_df['summary_text_standard'] = test_df['summary'].apply(lambda x: textstat.text_standard(x))

In [ ]:
#16.3
df_reviewerIDavg = df.groupby(['reviewerID']).mean()['rating'].reset_index()
df_reviewerIDavg.columns = ['reviewerID','reviewerratingavg']

test_df_reviewerIDavg = test_df.groupby(['reviewerID']).mean()['rating'].reset_index()
test_df_reviewerIDavg.columns = ['reviewerID','reviewerratingavg']

df_itemIDavg = df.groupby(['itemID']).mean()['rating'].reset_index()
df_itemIDavg.columns = ['itemID','itemratingavg']

test_df_itemIDavg = test_df.groupby(['itemID']).mean()['rating'].reset_index()
test_df_itemIDavg.columns = ['itemID','itemratingavg']

df = df.merge(df_reviewerIDavg,how='left',left_on='reviewerID',right_on='reviewerID')
df = df.merge(df_itemIDavg,how='left',left_on='itemID',right_on='itemID')

test_df = test_df.merge(df_reviewerIDavg,how='left',left_on='reviewerID',right_on='reviewerID')
test_df = test_df.merge(df_itemIDavg,how='left',left_on='itemID',right_on='itemID')

In [ ]:
df['reviewerratingratio'] = df['rating']/df['reviewerratingavg']
df['itemratingratio'] = df['rating']/df['itemratingavg']

test_df['reviewerratingratio'] = test_df['rating']/test_df['reviewerratingavg']
test_df['itemratingratio'] = test_df['rating']/test_df['itemratingavg']

In [ ]:
df.head()

In [ ]:
df1 = df['rating','outOf','reviewWords', 'summaryWords','summaryCwords', 'ratiosuWord', 'reviewSentences', 'summarySentences',
   'reviewChars', 'reviewRead','reviewPuncts', 'ratiopunChar', 'reviewCwords', 'ratioCwords',
   'summaryCwords', 'suratioCwords', 'reexcqueMarks', 'suexcqueMarks','numreviewPro', 'numreviews',
   'reviewerratingavg', 'itemratingavg','reviewerratingratio', 'itemratingratio', 'summarypolarity',
   'summarysubjectivity', 'reviewTextpolarity', 'reviewTextsubjectivity','reviewText_flesch_reading_ease',
   'reviewText_smog_index','reviewText_flesch_kincaid_grade', 'reviewText_coleman_liau_index',
   'reviewText_automated_readability_index','reviewText_dale_chall_readability_score',
   'reviewText_difficult_word','reviewText_linsear_write_formula','summary_flesch_reading_ease',
   'summary_smog_index','summary_flesch_kincaid_grade', 'summary_coleman_liau_index',
   'summary_automated_readability_index', 'summary_dale_chall_readability_score',
   'summary_difficult_word','summary_linsear_write_formula','reviewText_gunning_fogcat',
   'summary_gunning_fogcat']

In [ ]:
test_df1 = test_df['rating','outOf','reviewWords','summaryCwords', 'summaryWords', 'ratiosuWord', 'reviewSentences', 'summarySentences',
   'reviewChars', 'reviewRead','reviewPuncts', 'ratiopunChar', 'reviewCwords', 'ratioCwords',
   'summaryCwords', 'suratioCwords', 'reexcqueMarks', 'suexcqueMarks','numreviewPro', 'numreviews',
   'reviewerratingavg', 'itemratingavg','reviewerratingratio', 'itemratingratio', 'summarypolarity',
   'summarysubjectivity', 'reviewTextpolarity', 'reviewTextsubjectivity','reviewText_flesch_reading_ease',
   'reviewText_smog_index','reviewText_flesch_kincaid_grade', 'reviewText_coleman_liau_index',
   'reviewText_automated_readability_index','reviewText_dale_chall_readability_score',
   'reviewText_difficult_word','reviewText_linsear_write_formula','summary_flesch_reading_ease',
   'summary_smog_index','summary_flesch_kincaid_grade', 'summary_coleman_liau_index',
   'summary_automated_readability_index', 'summary_dale_chall_readability_score',
   'summary_difficult_word','summary_linsear_write_formula','reviewText_gunning_fogcat',
   'summary_gunning_fogcat']

In [ ]:
#'rating','helpful_denominator', 'reviewWords','summaryCwords',
#'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [723]:
#17. Breaking up helpful column into two features
df['helpful_numerator'] = df['helpful'].apply(lambda x: x['nHelpful'])

df['helpful_denominator'] = df['helpful'].apply(lambda x: x['outOf'])
test_df['helpful_denominator'] = test_df['helpful'].apply(lambda x: x['outOf'])

df['helpratio'] = df['helpful_numerator'] / df['helpful_denominator']
df['helpratio'] = df['helpratio'].fillna(0)
test_df.head(5)

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,...,reviewTextsubjectivity,reviewText_flesch_reading_ease,reviewText_smog_index,reviewText_coleman_liau_index,reviewText_automated_readability_index,reviewText_dale_chall_readability_score,reviewText_difficult_word,reviewText_linsear_write_formula,reviewText_gunning_fog,helpful_denominator
0,0,"[[Sports & Outdoors, Other Sports, Dance, Clot...",I520932398,U816789534,3.0,I ordered according to the size chart but it's...,R157684793,2011-07-15,Too small,1310688000,...,0.505000,83.15,0.0,7.07,6.4,7.23,5,6.750000,8.36,2
1,0,"[[Sports & Outdoors, Clothing, Women, Hoodies]...",I969532331,U987148846,4.0,Super thin but really cute and not cheap-looki...,R732719858,2013-07-17,Fun hoodie,1374019200,...,0.703333,69.45,0.0,7.09,12.5,6.73,3,6.750000,10.80,0
2,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I149943341,U628436634,5.0,"It was a present for my sis, and she loves Fle...",R352659313,2013-12-08,A Perfect hook up,1386460800,...,0.500000,81.97,0.0,4.71,8.3,1.83,1,5.250000,9.20,1
3,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I909025835,U924107228,5.0,I love this thing! I guess they don't make th...,R277416618,2012-11-22,I love this thing...,1353542400,...,0.367521,-31.72,0.0,5.60,64.5,11.50,10,17.333333,54.89,1
4,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I228439768,U060135484,4.0,I liked it and I wear it...it's a little bit s...,R645892076,2014-04-01,I liked it...,1396310400,...,0.677778,66.75,0.0,4.66,16.6,6.77,3,20.000000,16.25,0


In [724]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
#threshold = 0
#df['threshold'] = df['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df = df.loc[df['helpratio']> .15] #keep .5
#df = df[df['helpful_denominator']<=75] #keep 50
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
#df = df.loc[df['threshold'] > threshold]

In [725]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
#threshold = 0
#df54=df
#df54['threshold'] = df54['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df54['helpful_overall'] = np.where(df54.loc[:, 'helpratio']> .5, 1, 0) #keep .5
#df54 = df54[df54['helpful_denominator']>=2]
#df54 = df54[df54['helpful_denominator']<10]
#df54 = df54[df55['helpful_denominator']<50] #keep 50
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
#df54 = df54.loc[df54['threshold'] > threshold]

In [726]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
threshold = 0
df55=df
df55['threshold'] = df55['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
df55['helpful_overall'] = np.where(df55.loc[:, 'helpratio']> .5, 1, 0) #keep .5
df55 = df55[df55['helpful_denominator']>=2]
df55 = df55[df55['helpful_denominator']<50] #keep 50
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
df55 = df55.loc[df55['threshold'] > threshold]

In [727]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
threshold = 0
df56 = df
df56['threshold'] = df56['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df56['helpful_overall'] = np.where(df56.loc[:, 'helpratio']>= .10, 1, 0) #keep .5
df56 = df56[df56['helpful_denominator']>=50]
df56 = df56[df56['helpful_denominator']<80]#keep 100
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
df56 = df56.loc[df56['threshold'] > threshold]

In [728]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
#threshold = 0
#df57 = df
#df57['threshold'] = df57['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df57['helpful_overall'] = np.where(df57.loc[:, 'helpratio']>= .5, 1, 0) #keep .5
#df57 = df57[df57['helpful_denominator']>=100]
#df57 = df57[df57['helpful_denominator']<200]#keep 100
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
#df57 = df57.loc[df57['threshold'] > threshold]

In [729]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
#threshold = 0
#df58 = df
#df58['threshold'] = df58['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df58['helpful_overall'] = np.where(df58.loc[:, 'helpratio']>= .5, 1, 0) #keep .5
#df58 = df58[df58['helpful_denominator']>=200] #keep 100
#df58 = df58[df58['helpful_denominator']<300] #keep 100
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
#df58 = df58.loc[df58['threshold'] > threshold]

In [730]:
#Define helpful_overall as a binary variable with 0.75 helpratio as 1 o/w 0. Filter out those with 0 input.
#threshold = 0
#df59 = df
#df59['threshold'] = df59['helpful_denominator'].apply(lambda x: 1 if x>1 else 0)
#df59['helpful_overall'] = np.where(df59.loc[:, 'helpratio']>= .5, 1, 0) #keep .5
#df59 = df59[df59['helpful_denominator']>=200] #keep 100
#df1.loc[:, 'Helpful'] = np.where(df.loc[:, 'helpful_numerator'] / df.loc[:, 'helpful_denominator'] > threshold, 1, 0)
#df = df.loc[df['helpful_overall']==1]
#df59 = df59.loc[df59['threshold'] > threshold]

In [731]:
import xgboost as xgb
import random
from random import randint
from sklearn.model_selection import cross_val_predict, TimeSeriesSplit, KFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [732]:
#X54 = df54[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]
#y54 = df54['helpratio']

In [733]:
X55 = df55[['rating','helpful_denominator', 'reviewWords','summaryCwords', 'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]
y55 = df55['helpratio']

In [734]:
X56 = df56[['rating','helpful_denominator', 'reviewWords','summaryCwords', 'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]
y56 = df56['helpratio']

In [735]:
#X57 = df57[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords','reviewRead',
#                               'ratiosuWord','sunumexclquest','summarypolarity','summarysubjectivity','reviewTextpolarity','reviewTextsubjectivity', 'numreviewPro','numreviews']]
#y57 = df57['helpratio']

In [736]:
#X58 = df58[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]
#y58 = df58['helpratio']

In [737]:
#X59 = df59[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest','summarypolarity','summarysubjectivity','reviewTextpolarity','reviewTextsubjectivity', 'numreviewPro','numreviews']]
#y59 = df59['helpratio']

In [738]:
#from sklearn.model_selection import train_test_split
#X_train54, X_val54, y_train54, y_val54 = train_test_split(X54,y54, test_size=0.2, random_state=42)

In [739]:
from sklearn.model_selection import train_test_split
X_train55, X_val55, y_train55, y_val55 = train_test_split(X55,y55, test_size=0.2, random_state=42)

In [740]:
from sklearn.model_selection import train_test_split
X_train56, X_val56, y_train56, y_val56 = train_test_split(X56,y56, test_size=0.2, random_state=42)

In [741]:
#from sklearn.model_selection import train_test_split
#X_train57, X_val57, y_train57, y_val57 = train_test_split(X57,y57, test_size=0.2, random_state=42)

In [742]:
#from sklearn.model_selection import train_test_split
#X_train58, X_val58, y_train58, y_val58 = train_test_split(X58,y58, test_size=0.2, random_state=42)

In [743]:
#from sklearn.model_selection import train_test_split
#X_train59, X_val59, y_train59, y_val59 = train_test_split(X59,y59, test_size=0.2, random_state=42)

In [744]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.layers.recurrent import LSTM

In [745]:
#model54 = Sequential()
##get number of columns in training data
#n_cols54 = X_train54.shape[1]
##add model layers
#model54.add(LSTM(units = 105, input_shape=(X_train54.shape[1],1), return_sequences = True))
#model54.add(Dropout(0.2))
#model54.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model54.add(Dropout(0.2))
#model54.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model54.add(Dropout(0.2))
#model54.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model54.add(Dropout(0.2))
#model54.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model54.add(Dropout(0.2))
#model54.add(LSTM(units = 105))
#model54.add(Dropout(0.2))
#model54.add(Dense(units = 1, activation = 'sigmoid'))
#model54.compile(loss = 'mean_squared_error', optimizer = 'adam')
## reshape train data and value data to 3 dimension data structure
#X_train_list54 = np.reshape(np.array(X_train54),(X_train54.shape[0],X_train54.shape[1],1))
#X_val_list54 = np.reshape(np.array(X_val54), (X_val54.shape[0],X_val54.shape[1],1))
#y_train_list54 = np.array(y_train54)
#y_val_list54 = np.array(y_val54)
#model54.fit(X_train_list54, y_train_list54, epochs = 75)
#y_predict54 = model54.predict(X_val_list54)
#rms54 = np.sqrt(mean_squared_error(y_val_list54, y_predict54))
#print ('RMSE:', rms54)

In [746]:
model55 = Sequential()
#get number of columns in training data
n_cols55 = X_train55.shape[1]
#add model layers
model55.add(LSTM(units = 105, input_shape=(X_train55.shape[1],1), return_sequences = True))
model55.add(Dropout(0.2))
model55.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model55.add(Dropout(0.2))
model55.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model55.add(Dropout(0.2))
model55.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model55.add(Dropout(0.2))
model55.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model55.add(Dropout(0.2))
model55.add(LSTM(units = 105))
model55.add(Dropout(0.2))
model55.add(Dense(units = 1, activation = 'sigmoid'))
model55.compile(loss = 'mean_squared_error', optimizer = 'adam')
# reshape train data and value data to 3 dimension data structure
X_train_list55 = np.reshape(np.array(X_train55),(X_train55.shape[0],X_train55.shape[1],1))
X_val_list55 = np.reshape(np.array(X_val55), (X_val55.shape[0],X_val55.shape[1],1))
y_train_list55 = np.array(y_train55)
y_val_list55 = np.array(y_val55)
model55.fit(X_train_list55, y_train_list55, epochs = 75)
y_predict55 = model55.predict(X_val_list55)
rms55 = np.sqrt(mean_squared_error(y_val_list55, y_predict55))
print ('RMSE:', rms55)

Epoch 1/75
27252/27252 [==============================] - 22s 807us/step - loss: 0.0613
Epoch 2/75
27252/27252 [==============================] - 20s 725us/step - loss: 0.0588
Epoch 3/75
27252/27252 [==============================] - 20s 739us/step - loss: 0.0585
Epoch 4/75
27252/27252 [==============================] - 21s 765us/step - loss: 0.0583
Epoch 5/75
27252/27252 [==============================] - 21s 756us/step - loss: 0.0581
Epoch 6/75
27252/27252 [==============================] - 20s 740us/step - loss: 0.0582
Epoch 7/75
27252/27252 [==============================] - 20s 749us/step - loss: 0.0579
Epoch 8/75
27252/27252 [==============================] - 20s 747us/step - loss: 0.0578
Epoch 9/75
27252/27252 [==============================] - 20s 751us/step - loss: 0.0577
Epoch 10/75
27252/27252 [==============================] - 20s 750us/step - loss: 0.0577
Epoch 11/75
27252/27252 [==============================] - 20s 747us/step - loss: 0.0577
Epoch 12/75
27252/27252 [=====

In [747]:
model56 = Sequential()
#get number of columns in training data
n_cols56 = X_train56.shape[1]
#add model layers
model56.add(LSTM(units = 105, input_shape=(X_train56.shape[1],1), return_sequences = True))
model56.add(Dropout(0.2))
model56.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model56.add(Dropout(0.2))
model56.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model56.add(Dropout(0.2))
model56.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model56.add(Dropout(0.2))
model56.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
model56.add(Dropout(0.2))
model56.add(LSTM(units = 105))
model56.add(Dropout(0.2))
model56.add(Dense(units = 1, activation = 'sigmoid'))
model56.compile(loss = 'mean_squared_error', optimizer = 'adam')
# reshape train data and value data to 3 dimension data structure
X_train_list56 = np.reshape(np.array(X_train56),(X_train56.shape[0],X_train56.shape[1],1))
X_val_list56 = np.reshape(np.array(X_val56), (X_val56.shape[0],X_val56.shape[1],1))
y_train_list56 = np.array(y_train56)
y_val_list56 = np.array(y_val56)
model56.fit(X_train_list56, y_train_list56, epochs = 75)
y_predict56 = model56.predict(X_val_list56)
rms56 = np.sqrt(mean_squared_error(y_val_list56, y_predict56))
print ('RMSE:', rms56)

Epoch 1/75
236/236 [==============================] - 2s 9ms/step - loss: 0.1781
Epoch 2/75
236/236 [==============================] - 0s 768us/step - loss: 0.0448
Epoch 3/75
236/236 [==============================] - 0s 757us/step - loss: 0.0104
Epoch 4/75
236/236 [==============================] - 0s 753us/step - loss: 0.0103
Epoch 5/75
236/236 [==============================] - 0s 737us/step - loss: 0.0097
Epoch 6/75
236/236 [==============================] - 0s 734us/step - loss: 0.0086
Epoch 7/75
236/236 [==============================] - 0s 749us/step - loss: 0.0070
Epoch 8/75
236/236 [==============================] - 0s 752us/step - loss: 0.0073
Epoch 9/75
236/236 [==============================] - 0s 772us/step - loss: 0.0067
Epoch 10/75
236/236 [==============================] - 0s 768us/step - loss: 0.0068
Epoch 11/75
236/236 [==============================] - 0s 763us/step - loss: 0.0073
Epoch 12/75
236/236 [==============================] - 0s 777us/step - loss: 0.0071
Epo

In [748]:
#model57 = Sequential()
##get number of columns in training data
#n_cols57 = X_train57.shape[1]
##add model layers
#model57.add(LSTM(units = 105, input_shape=(X_train57.shape[1],1), return_sequences = True))
#model57.add(Dropout(0.2))
#model57.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model57.add(Dropout(0.2))
#model57.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model57.add(Dropout(0.2))
#model57.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model57.add(Dropout(0.2))
#model57.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model57.add(Dropout(0.2))
#model57.add(LSTM(units = 105))
#model57.add(Dropout(0.2))
#model57.add(Dense(units = 1, activation = 'sigmoid'))
#model57.compile(loss = 'mean_squared_error', optimizer = 'adam')
## reshape train data and value data to 3 dimension data structure
#X_train_list57 = np.reshape(np.array(X_train57),(X_train57.shape[0],X_train57.shape[1],1))
#X_val_list57 = np.reshape(np.array(X_val57), (X_val57.shape[0],X_val57.shape[1],1))
#y_train_list57 = np.array(y_train57)
#y_val_list57 = np.array(y_val57)
#model57.fit(X_train_list57, y_train_list57, epochs = 75)
#y_predict57 = model57.predict(X_val_list57)
#rms57 = np.sqrt(mean_squared_error(y_val_list57, y_predict57))
#print ('RMSE:', rms57)

In [749]:
#model58 = Sequential()
##get number of columns in training data
#n_cols58 = X_train58.shape[1]
##add model layers
#model58.add(LSTM(units = 105, input_shape=(X_train58.shape[1],1), return_sequences = True))
#model58.add(Dropout(0.2))
#model58.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model58.add(Dropout(0.2))
#model58.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model58.add(Dropout(0.2))
#model58.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model58.add(Dropout(0.2))
#model58.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model58.add(Dropout(0.2))
#model58.add(LSTM(units = 105))
#model58.add(Dropout(0.2))
#model58.add(Dense(units = 1, activation = 'sigmoid'))
#model58.compile(loss = 'mean_squared_error', optimizer = 'adam')
## reshape train data and value data to 3 dimension data structure
#X_train_list58 = np.reshape(np.array(X_train58),(X_train58.shape[0],X_train58.shape[1],1))
#X_val_list58 = np.reshape(np.array(X_val58), (X_val58.shape[0],X_val58.shape[1],1))
#y_train_list58 = np.array(y_train58)
#y_val_list58 = np.array(y_val58)
#model58.fit(X_train_list58, y_train_list58, epochs = 75)
#y_predict58 = model58.predict(X_val_list58)
#rms58 = np.sqrt(mean_squared_error(y_val_list58, y_predict58))
#print ('RMSE:', rms58)

In [750]:
#model59 = Sequential()
##get number of columns in training data
#n_cols59 = X_train59.shape[1]
##add model layers
#model59.add(LSTM(units = 105, input_shape=(X_train59.shape[1],1), return_sequences = True))
#model59.add(Dropout(0.2))
#model59.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model59.add(Dropout(0.2))
#model59.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model59.add(Dropout(0.2))
#model59.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model59.add(Dropout(0.2))
#model59.add(LSTM(units = 105, return_sequences = True, activation = 'relu'))
#model59.add(Dropout(0.2))
#model59.add(LSTM(units = 105))
#model59.add(Dropout(0.2))
#model59.add(Dense(units = 1, activation = 'sigmoid'))
#model59.compile(loss = 'mean_squared_error', optimizer = 'adam')
## reshape train data and value data to 3 dimension data structure
#X_train_list59 = np.reshape(np.array(X_train59),(X_train59.shape[0],X_train59.shape[1],1))
#X_val_list59 = np.reshape(np.array(X_val59), (X_val59.shape[0],X_val59.shape[1],1))
#y_train_list59 = np.array(y_train59)
#y_val_list59 = np.array(y_val59)
#model59.fit(X_train_list59, y_train_list59, epochs = 75)
#y_predict59 = model59.predict(X_val_list59)
#rms59 = np.sqrt(mean_squared_error(y_val_list59, y_predict59))
#print ('RMSE:', rms59)

In [751]:
#final_df =test_df[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [752]:
#final_df54 = test_df[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [753]:
final_df55 = test_df[['rating','helpful_denominator', 'reviewWords','summaryCwords', 'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [754]:
final_df56 = test_df[['rating','helpful_denominator', 'reviewWords','summaryCwords', 'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [755]:
#final_df57 = test_df[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords','reviewRead',
#                               'ratiosuWord','sunumexclquest','summarypolarity','summarysubjectivity','reviewTextpolarity','reviewTextsubjectivity', 'numreviewPro','numreviews']]

In [756]:
#final_df58 = test_df[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest', 'numreviewPro','numreviews']]

In [757]:
#final_df59 = test_df[['rating','unixReviewTime','helpful_denominator', 'reviewWords','summaryCwords',
#                               'ratiosuWord','sunumexclquest','summarypolarity','summarysubjectivity','reviewTextpolarity','reviewTextsubjectivity', 'numreviewPro','numreviews']]

In [758]:
#X_final_54 = np.reshape(np.array(final_df54),(final_df54.shape[0],final_df54.shape[1],1))

In [759]:
X_final_55 = np.reshape(np.array(final_df55),(final_df55.shape[0],final_df55.shape[1],1))

In [760]:
X_final_56 = np.reshape(np.array(final_df56),(final_df56.shape[0],final_df56.shape[1],1))

In [761]:
#X_final_57 = np.reshape(np.array(final_df57),(final_df57.shape[0],final_df57.shape[1],1))

In [762]:
#X_final_58 = np.reshape(np.array(final_df58),(final_df58.shape[0],final_df58.shape[1],1))

In [763]:
#X_final_59 = np.reshape(np.array(final_df59),(final_df59.shape[0],final_df59.shape[1],1))

In [764]:
#test_df['votingprob54'] = model55.predict(X_final_54)

In [765]:
test_df['votingprob55'] = model55.predict(X_final_55)

In [766]:
test_df['votingprob56'] = model56.predict(X_final_56)

In [767]:
#test_df['votingprob57'] = model57.predict(X_final_57)

In [768]:
#test_df['votingprob58'] = model55.predict(X_final_58)

In [769]:
#test_df['votingprob59'] = model55.predict(X_final_59)

In [770]:
counter_test = 0
prediction_values = []
for i in test_df['helpful_denominator']:
    if i == 0:
        prediction_values.append(0)
    elif i == 1:
        if test_df.loc[counter_test,['rating']].item() >=3:
            prediction_values.append(1)
        else:
            prediction_values.append(0)
    elif i == 2:
        if test_df.loc[counter_test,['rating']].item() >=3:
            prediction_values.append(2)
        else:
            prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob55'],0)))
    elif test_df.loc[counter_test,['rating']].item() ==5:
        prediction_values.append(i)
    elif test_df.loc[counter_test,['rating']].item() ==4:
        prediction_values.append(i)
    #elif test_df.loc[counter_test,['rating']].item() ==1:
    #    if probas3[counter_test][1] < .5:
    #        prediction_values.append(0)
    #    else:
    #        prediction_values.append(int(round(i*probas3[counter_test][1],0)))
    #elif probas3[counter_test][1] <= .5:
    #    prediction_values.append(int(round(i*probas3[counter_test][0],0)))
    #elif probas3[counter_test][1] <=.65:
        #print(probas3[counter_test][1])
    #    prediction_values.append(int(round(i*probas3[counter_test][1],0)))
    #elif test_df.loc[counter_test,['helpful_denominator']].item() >= 200:
    #    prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob59'],0)))
    #elif 300 > test_df.loc[counter_test,['helpful_denominator']].item() >= 200:
    #    prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob58'],0)))
    #elif 200 > test_df.loc[counter_test,['helpful_denominator']].item() >= 100:
   #     prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob57'],0)))
    elif 100 > test_df.loc[counter_test,['helpful_denominator']].item() >= 50:
        prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob56'],0)))
    #elif 50 > test_df.loc[counter_test,['helpful_denominator']].item() >= 10:
    #    prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob55'],0)))
    else:
        prediction_values.append(int(round(i*test_df.loc[counter_test,'votingprob55'],0)))
    counter_test += 1
prediction_values

/Users/felipegomez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/felipegomez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys
/Users/felipegomez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/felipegomez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/felipegomez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: `item` has been deprecated and will be removed in a future version


[2,
 0,
 1,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 7,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 9,
 0,
 0,
 9,
 0,
 5,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 10,
 1,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 17,
 1,
 0,
 0,
 0,
 2,
 8,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 3,
 3,
 2,
 81,
 2,
 1,
 0,
 1,
 6,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 95,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 4,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 2,
 1,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 2,
 17,
 1,
 2,
 0,
 5,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [771]:
len(prediction_values)

14000

In [772]:
#test_df.drop(columns=['predictions'])
test_df['predictions'] = prediction_values
test_df.head()

,categoryID,categories,itemID,reviewerID,rating,reviewText,reviewHash,reviewTime,summary,unixReviewTime,...,reviewText_coleman_liau_index,reviewText_automated_readability_index,reviewText_dale_chall_readability_score,reviewText_difficult_word,reviewText_linsear_write_formula,reviewText_gunning_fog,helpful_denominator,votingprob55,votingprob56,predictions
0,0,"[[Sports & Outdoors, Other Sports, Dance, Clot...",I520932398,U816789534,3.0,I ordered according to the size chart but it's...,R157684793,2011-07-15,Too small,1310688000,...,7.07,6.4,7.23,5,6.750000,8.36,2,0.756702,0.939719,2
1,0,"[[Sports & Outdoors, Clothing, Women, Hoodies]...",I969532331,U987148846,4.0,Super thin but really cute and not cheap-looki...,R732719858,2013-07-17,Fun hoodie,1374019200,...,7.09,12.5,6.73,3,6.750000,10.80,0,0.607543,0.940342,0
2,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I149943341,U628436634,5.0,"It was a present for my sis, and she loves Fle...",R352659313,2013-12-08,A Perfect hook up,1386460800,...,4.71,8.3,1.83,1,5.250000,9.20,1,0.765562,0.938215,1
3,0,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",I909025835,U924107228,5.0,I love this thing! I guess they don't make th...,R277416618,2012-11-22,I love this thing...,1353542400,...,5.60,64.5,11.50,10,17.333333,54.89,1,0.781770,0.937813,1
4,0,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",I228439768,U060135484,4.0,I liked it and I wear it...it's a little bit s...,R645892076,2014-04-01,I liked it...,1396310400,...,4.66,16.6,6.77,3,20.000000,16.25,0,0.666313,0.939687,0


In [773]:
counter = 0
predictions = open("predictions_Helpful_testdfLSTM36.txt", 'w')
for l in open("pairs_Helpful.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i,outOf = l.strip().split('-')
  outOf = int(outOf)
  #for pred in b:
  predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(test_df.loc[counter,'predictions']) + '\n')
  counter +=1
  #else:
  #  predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')
predictions.close()

In [774]:
#Keep 19 and 18 so far for Final Submission.